# DIRE Detection Jupyter Notebook
This notebook implements the DIffusion REconstruction Error (DIRE) detector for distinguishing real vs. diffusion-generated images.

In [1]:

%pip install diffusers

import torch
import torch.nn.functional as F
from diffusers import DDPMPipeline, DDPMScheduler
from torchvision import transforms
from PIL import Image

# Configuration
device = "cuda" if torch.cuda.is_available() else "cpu"
model_id = "google/ddpm-cifar10-32"
num_inference_steps = 100
dire_threshold = 0.01

# Load pipeline and scheduler
pipe = DDPMPipeline.from_pretrained(model_id, torch_dtype=torch.float16).to(device)
scheduler = DDPMScheduler.from_pretrained(model_id)
scheduler.set_timesteps(num_inference_steps)

# Preprocessing
preprocess = transforms.Compose([
    transforms.Resize((32, 32)),
    transforms.ToTensor(),
])

def compute_dire(img: Image.Image):
    x0 = preprocess(img).unsqueeze(0).to(device)
    t = scheduler.timesteps[-1]
    noise = torch.randn_like(x0)
    xt = scheduler.add_noise(x0, noise, t)
    x_recon = xt
    for t in scheduler.timesteps:
        with torch.no_grad():
            model_output = pipe.unet(x_recon, t)["sample"]
        x_recon = scheduler.step(model_output, t, x_recon)["prev_sample"]
    dire = F.mse_loss(x_recon, x0).item()
    return dire, x_recon.squeeze(0).cpu()

def detect_via_dire(img: Image.Image):
    dire, recon = compute_dire(img)
    is_diffusion = dire < dire_threshold
    return is_diffusion, dire, recon


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.

[notice] A new release of pip is available: 23.2.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


  Obtaining dependency information for diffusers from https://files.pythonhosted.org/packages/f8/e0/d5af850081d479e5bb6f6f310e98e1e2ea6cce9e5d67e2b7978d5690497e/diffusers-0.34.0-py3-none-any.whl.metadata
  Obtaining dependency information for importlib_metadata from https://files.pythonhosted.org/packages/20/b0/36bd937216ec521246249be3bf9855081de4c5e06a0c9b4219dbeda50373/importlib_metadata-8.7.0-py3-none-any.whl.metadata
  Obtaining dependency information for filelock from https://files.pythonhosted.org/packages/4d/36/2a115987e2d8c300a974597416d9de88f2444426de9571f4b59b2cca3acc/filelock-3.18.0-py3-none-any.whl.metadata
  Obtaining dependency information for huggingface-hub>=0.27.0 from https://files.pythonhosted.org/packages/44/f4/5f3f22e762ad1965f01122b42dae5bf0e009286e2dba601ce1d0dba72424/huggingface_hub-0.33.2-py3-none-any.whl.metadata
  Obtaining dependency information for numpy from https://files.pythonhosted.org/packages/b1/3e/e28f4c1dd9e042eb57a3eb652f200225e311b608632bc727ae378

ModuleNotFoundError: No module named 'torch'

In [ ]:

# Example usage
if __name__ == "__main__":
    import sys
    if len(sys.argv) != 2:
        print("Usage: python detect_dire.py path/to/image.png")
        sys.exit(1)
    img = Image.open(sys.argv[1]).convert("RGB")
    is_diff, dire_score, recon_img = detect_via_dire(img)
    print(f"DIRE score = {dire_score:.6f}")
    print("Detected as", "diffusion-generated" if is_diff else "real")
    recon_pil = transforms.ToPILImage()(recon_img)
    recon_pil.save("reconstruction.png")
